In [99]:
import pandas as pd
import geopandas as gpd
import censusdata
import numpy as np
import warnings; warnings.simplefilter('ignore')

In [100]:
#censusdata.search('acs5', 2015, 'concept', 'age')

In [101]:
df = censusdata.download('acs5', 2015, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001A_013E', 'B01001A_014E', 'B01001A_015E', 'B01001A_016E', 'B01001A_028E',
                              'B01001A_029E', 'B01001A_030E', 'B01001A_031E', 'B01001B_007E','B01001B_008E',
                              'B01001B_009E','B01001B_022E','B01001B_023E','B01001B_024E','B01001C_007E',
                              'B01001C_008E','B01001C_009E','B01001C_022E','B01001C_023E','B01001C_024E',
                              'B01001D_007E','B01001D_008E','B01001D_009E','B01001D_022E','B01001D_023E',
                              'B01001D_024E'])
sum_column = df['B01001A_013E'] + df['B01001A_014E'] + df['B01001A_015E'] + df['B01001A_016E'] +  df['B01001A_028E'] + df['B01001A_029E'] + df['B01001A_030E'] + df['B01001A_031E']
df['Total White 55+'] = sum_column
sum_column = df[['B01001B_007E','B01001B_008E','B01001B_009E','B01001B_022E','B01001B_023E','B01001B_024E',
                'B01001C_007E','B01001C_008E','B01001C_009E','B01001C_022E','B01001C_023E','B01001C_024E',
                'B01001D_007E','B01001D_008E','B01001D_009E','B01001D_022E','B01001D_023E','B01001D_024E']].sum(axis=1)
df['Total Non-White 18-29'] = sum_column
df = df.reset_index(drop=False)
df = df[['index', 'Total White 55+', 'Total Non-White 18-29']]
df.rename(columns = {'index' : 'TRACT'}, inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df15 = df.astype({"TRACT": int})

#df15



In [102]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001A_007E', 'B01001A_008E', 'B01001A_009E'])
sum_column = df['B01001A_007E'] + df['B01001A_008E'] + df['B01001A_009E']
df['Total White 18-29'] = sum_column
df = df.reset_index(drop=False)
df = df[['index', 'Total White 18-29']]
df.rename(columns = {'index' : 'TRACT'}, inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
df = df.astype({"TRACT": int})
    
#display(df)

    
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)
                
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)

temp = pd.read_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2011-15.csv')
        
neigh_age_11_15.insert(1, 'Total', temp['TOTAL'])

temp = neigh_age_11_15.copy()

cols = list(neigh_age_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_11_15)):
        neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)

neigh_age_11_15.rename(columns={'Total White 18-29' : '% White 18-29'}, inplace=True)
neigh_age_11_15['Total White 18-29'] = temp['Total White 18-29']      

neigh_age_11_15 = neigh_age_11_15.drop(25)
neigh_age_11_15

,NEIGHBORHOOD,Total,% White 18-29,Total White 18-29
0,Roslindale,29024,0.041,1189
1,Jamaica Plain,42759,0.058,2496
2,Mission Hill,13920,0.184,2561
3,Longwood,6241,0.206,1288
4,Bay Village,1399,0.074,104
5,Leather District,537,0.134,72
6,Chinatown,2718,0.128,349
7,North End,6964,0.187,1301
8,Roxbury,50601,0.044,2202
9,South End,23667,0.071,1679


In [103]:
neigh_age_11_15.to_csv('Boston_Neighborhoods_White_18-29_Population_2015-2019.csv')

In [104]:
temp = pd.read_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2015-19.csv')

df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001A_013E', 'B01001A_014E', 'B01001A_015E', 'B01001A_016E', 'B01001A_028E',
                              'B01001A_029E', 'B01001A_030E', 'B01001A_031E', 'B01001B_007E','B01001B_008E',
                              'B01001B_009E','B01001B_022E','B01001B_023E','B01001B_024E','B01001C_007E',
                              'B01001C_008E','B01001C_009E','B01001C_022E','B01001C_023E','B01001C_024E',
                              'B01001D_007E','B01001D_008E','B01001D_009E','B01001D_022E','B01001D_023E',
                              'B01001D_024E'])
sum_column = df['B01001A_013E'] + df['B01001A_014E'] + df['B01001A_015E'] + df['B01001A_016E'] +  df['B01001A_028E'] + df['B01001A_029E'] + df['B01001A_030E'] + df['B01001A_031E']
df['Total White 55+'] = sum_column
sum_column = df[['B01001B_007E','B01001B_008E','B01001B_009E','B01001B_022E','B01001B_023E','B01001B_024E',
                'B01001C_007E','B01001C_008E','B01001C_009E','B01001C_022E','B01001C_023E','B01001C_024E',
                'B01001D_007E','B01001D_008E','B01001D_009E','B01001D_022E','B01001D_023E','B01001D_024E']].sum(axis=1)
df['Total Non-White 18-29'] = sum_column
df = df.reset_index(drop=False)
df = df[['index', 'Total White 55+', 'Total Non-White 18-29']]
df.rename(columns = {'index' : 'TRACT'}, inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df19 = df.astype({"TRACT": int})

#df19


In [105]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df15['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df15[filt][c] * dec)
                
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)

temp = pd.read_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2011-15.csv')
        
neigh_age_11_15.insert(1, 'Total', temp['TOTAL'])

temp = neigh_age_11_15.copy()

cols = list(neigh_age_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_11_15)):
        neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)

neigh_age_11_15.rename(columns={'Total White 55+' : '% White 55+', 'Total Non-White 18-29' : '% POC 18-29'},
                      inplace=True)
neigh_age_11_15['Total White 55+'] = temp['Total White 55+']
neigh_age_11_15['Total POC 18-29'] = temp['Total Non-White 18-29']        

neigh_age_11_15

,NEIGHBORHOOD,Total,% White 55+,% POC 18-29,Total White 55+,Total POC 18-29
0,Roslindale,29024,0.172,0.058,4989,1677
1,Jamaica Plain,42759,0.127,0.052,5412,2220
2,Mission Hill,13920,0.074,0.134,1031,1867
3,Longwood,6241,0.036,0.145,225,905
4,Bay Village,1399,0.147,0.055,205,77
5,Leather District,537,0.058,0.117,31,63
6,Chinatown,2718,0.067,0.110,182,300
7,North End,6964,0.188,0.020,1312,142
8,Roxbury,50601,0.028,0.132,1404,6691
9,South End,23667,0.117,0.063,2759,1497


In [106]:
neigh_age_11_15.to_csv('Old_White_Pop_VS_Young_POC_Pop_11-15.csv')

In [107]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df19['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df19[filt][c] * dec)
                
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_age_15_19 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_15_19.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_15_19)):
    neighb = neigh_age_15_19['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_15_19[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_15_19.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_15_19[c] = neigh_age_15_19[c].round().astype(int)

temp = pd.read_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2015-19.csv')
        
neigh_age_15_19.insert(1, 'Total', temp['TOTAL'])        

cols = list(neigh_age_15_19.columns)[2:]

temp = neigh_age_15_19.copy()

for c in cols:
    for i in range(len(neigh_age_15_19)):
        neigh_age_15_19.loc[i, c] = round(neigh_age_15_19.loc[i, c]/neigh_age_15_19.loc[i, 'Total'], 3)
        
neigh_age_15_19.rename(columns={'Total White 55+' : '% White 55+', 'Total Non-White 18-29' : '% POC 18-29'},
                      inplace=True)
neigh_age_15_19['Total White 55+'] = temp['Total White 55+']
neigh_age_15_19['Total POC 18-29'] = temp['Total Non-White 18-29']

neigh_age_15_19

,NEIGHBORHOOD,Total,% White 55+,% POC 18-29,Total White 55+,Total POC 18-29
0,Roslindale,30207,0.187,0.046,5642,1390
1,Jamaica Plain,43871,0.158,0.045,6950,1967
2,Mission Hill,14251,0.082,0.167,1162,2375
3,Longwood,6549,0.036,0.189,239,1237
4,Bay Village,1465,0.154,0.048,225,70
5,Leather District,621,0.071,0.077,44,48
6,Chinatown,3113,0.079,0.075,245,232
7,North End,6785,0.184,0.020,1251,136
8,Roxbury,53772,0.038,0.130,2063,7013
9,South End,24202,0.129,0.047,3112,1145


In [108]:
neigh_age_15_19.to_csv('Old_White_Pop_VS_Young_POC_Pop_15-19.csv')